In [0]:
# Databricks Notebook: Transform Bronze (Fear & Greed) -> Silver
# Binance용과 동일한 패턴: 최근 N일 증분 + unique_key 기준 MERGE

from pyspark.sql.functions import col
from delta.tables import DeltaTable

# ===== (0) 선택: 최근 N일만 증분 처리 =====
DAYS_BACK = 60  # FNG는 일 1회 수준이라 넉넉히

# =========================
# (A) 환경/대상 테이블 설정
# =========================
CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
BRONZE  = f"{CATALOG}.{SCHEMA}.bronze_fear_greed"
SILVER  = f"{CATALOG}.{SCHEMA}.silver_fear_greed"

# =========================
# (B) Silver 테이블 생성(없으면)
# =========================
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {SILVER} (
  event_time           TIMESTAMP,
  index_value          INT,
  value_classification STRING,
  time_until_update    STRING,
  unique_key           STRING,   -- "fear_greed|<unix_ts>"
  dt                   DATE
) USING DELTA
PARTITIONED BY (dt)
""")

# =========================
# (C) Bronze 로드(증분 범위 선택) -> 정형 DF
# =========================
bronze = spark.table(BRONZE)
if DAYS_BACK is not None:
    bronze = bronze.where(f"dt >= date_sub(current_date(), {DAYS_BACK})")

silver_df = (
    bronze
      .select(
          col("event_time").alias("event_time"),
          col("index_value").cast("int").alias("index_value"),
          col("value_classification").alias("value_classification"),
          col("time_until_update").alias("time_until_update"),
          col("unique_key").alias("unique_key"),
          col("dt").cast("date").alias("dt"),
      )
      .dropDuplicates(["unique_key"])
      .repartition("dt")  # 선택: 파티션별 MERGE 최적화
)

# =========================
# (D) Delta MERGE (idempotent upsert)
# =========================
target = DeltaTable.forName(spark, SILVER)

(target.alias("t")
  .merge(
    silver_df.alias("s"),
    "t.unique_key = s.unique_key AND t.dt = s.dt"
  )
  .whenMatchedUpdate(set={
      "event_time":           "s.event_time",
      "index_value":          "s.index_value",
      "value_classification": "s.value_classification",
      "time_until_update":    "s.time_until_update",
      "dt":                   "s.dt"
  })
  .whenNotMatchedInsertAll()
  .execute())

print(f"Silver FNG transform complete: upserted into {SILVER}")
